<H1> 1.Logistic regression</H1>

In [2]:
#import requird libraries
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
# Load the penguins dataset
df = sns.load_dataset("penguins")
df.dropna(inplace=True)


In [4]:
# Filter rows for 'Adelie' and 'Chinstrap' classes
selected_classes = ['Adelie', 'Chinstrap']
df_filtered = df[df['species'].isin(selected_classes)].copy()  # Make a copy to avoid the warning

In [5]:
# Initialize the LabelEncoder
le = LabelEncoder()

In [6]:
# Encode the species column
y_encoded = le.fit_transform(df_filtered['species'])
df_filtered['class_encoded'] = y_encoded

<p><strong>Q2. What is the purpose of "y_encoded = le.fit_transform(df_filtered['species'])" ?</strong></p>
<p>Ans: To convert the categorical values in the 'species' column into numeric labels.</p>
<p>e.g., 0 for 'Adelie' and 1 for 'Chinstrap'</p>


In [7]:
# Display the filtered and encoded DataFrame
print(df_filtered[['species', 'class_encoded']])

       species  class_encoded
0       Adelie              0
1       Adelie              0
2       Adelie              0
4       Adelie              0
5       Adelie              0
..         ...            ...
215  Chinstrap              1
216  Chinstrap              1
217  Chinstrap              1
218  Chinstrap              1
219  Chinstrap              1

[214 rows x 2 columns]


In [8]:
# Split the data into features (X) and target variable (y)
y = df_filtered['class_encoded']  # Target variable
X = df_filtered.drop(['species', 'island', 'sex', 'class_encoded'], axis=1)

<p><strong>Q3. What is the purpose of "X = df.drop(['species', 'island', 'sex'], axis=1)" ?</strong></p>
<p>Ans: To remove unnecessary or non-numeric columns from the dataset that are not used as features for the model.</p>


<p><strong>Q4. Why can't we use "island" and "sex" features?</strong></p>
<p>Ans: These are not numerical data. Features need to be numerical to work with the model.</p>


In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<p><strong>Q6. What is the usage of "random_state=42" ?</strong></p>
<p>Ans: The same random split will occur each time, ensuring that your results are consistent. Without specifying this, the splits could differ across runs.</p>


In [10]:
# Train the logistic regression model with saga solver
logreg = LogisticRegression(solver='saga')
logreg.fit(X_train, y_train)

c:\Python312\cv\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(solver='saga')

In [11]:
# Check the number of data points in each class in the filtered DataFrame
class_counts = df_filtered['species'].value_counts()
print(class_counts)


species
Adelie       146
Chinstrap     68
Name: count, dtype: int64


<p><strong>Q7.</strong></p>
<p><strong>i) Why is accuracy low?</strong></p>
<p>Ans: Class Imbalance - Adelie has 146 & Chinstrap has only 68 data points.<br>
   Insufficient or Non-Informative Features.</p>

<p><strong>ii) Why does the saga solver perform poorly?</strong></p>
<p>Ans: The saga solver is typically used for large datasets and sparse data because it is efficient for dealing with such cases. However, on small datasets, it may not perform as well.<br>
   Also there is a convergence Issues.</p>


In [12]:
# Predict on the testing data
y_pred = logreg.predict(X_test)


In [13]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(logreg.coef_, logreg.intercept_)

Accuracy: 0.5813953488372093
[[ 2.74923636e-03 -8.39004562e-05  4.43377474e-04 -2.85413636e-04]] [-8.59398029e-06]


In [14]:
# Train the logistic regression model with saga solver
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
# Predict on the testing data
y_pred = logreg.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Q8. Accuracy with liblinear:", accuracy *100 , "%")
print(logreg.coef_, logreg.intercept_)

Q8. Accuracy with liblinear: 100.0 %
[[ 1.59665154 -1.42501103 -0.15238046 -0.003951  ]] [-0.0755452]


<p><strong>Q9. Why does the "liblinear" solver perform better than "saga" solver?</strong></p>
<p>Ans: Liblinear is specifically optimized for smaller datasets and works well with binary classification problems.<br>
   Liblinear is suitable for dense data while saga is better suited for large, sparse datasets.</p>


<p><strong>Q10. Compare the performance of the "liblinear" and "saga" solvers with feature scaling.</strong></p>

In [15]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train and evaluate liblinear solver
logreg_liblinear = LogisticRegression(solver='liblinear')
logreg_liblinear.fit(X_train_scaled, y_train)
y_pred_liblinear = logreg_liblinear.predict(X_test_scaled)
accuracy_liblinear = accuracy_score(y_test, y_pred_liblinear)

# Train and evaluate saga solver
logreg_saga = LogisticRegression(solver='saga')
logreg_saga.fit(X_train_scaled, y_train)
y_pred_saga = logreg_saga.predict(X_test_scaled)
accuracy_saga = accuracy_score(y_test, y_pred_saga)

# Print accuracies
print("Accuracy (liblinear): {:.2f}%".format(accuracy_liblinear * 100))
print("Accuracy (saga): {:.2f}%".format(accuracy_saga * 100))


Accuracy (liblinear): 97.67%
Accuracy (saga): 97.67%



<p>Ans: The saga solver shows a more pronounced improvement.</p>
<p><strong>Reasons:</strong></p>
<ul>
    <li><strong>Gradient Descent Efficiency:</strong> The saga solver uses a stochastic gradient descent approach, which benefits significantly from feature scaling.</li>
    <li><strong>Balanced Regularization Impact:</strong> The saga solver is designed to handle both L1 (Lasso) and L2 (Ridge) regularization effectively.</li>
</ul>


<H3> Logistic regression with multiple features.</H3>

In [16]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the penguins dataset
df = sns.load_dataset("penguins")
df.dropna(inplace=True)

# Filter rows for 'Adelie' and 'Chinstrap' classes
selected_classes = ['Adelie', 'Chinstrap']
df_filtered = df[df['species'].isin(selected_classes)].copy()  # Make a copy to avoid the warning

# Initialize the LabelEncoder
le = LabelEncoder()

# Encode the species column
y_encoded = le.fit_transform(df_filtered['species'])
df_filtered['class_encoded'] = y_encoded

df_filtered.head()

X = df_filtered.drop(['species', 'class_encoded'], axis=1)
y = df_filtered['class_encoded']  # Target variable
X.head()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression(solver='saga')
logreg.fit(X_train, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(logreg.coef_, logreg.intercept_)


ValueError: could not convert string to float: 'Dream'

<p><strong>Q11. What is the problem of this code and how to solve this?</strong></p>
<p>Ans: Non-Numeric Data: Features in X contain non-numeric columns (like categorical variables), which the LogisticRegression model cannot process.</p>
<p><strong>Solution:</strong> Convert categorical variables to numeric using one-hot encoding or label encoding to ensure all columns in X are numeric before fitting the model.</p>


In [37]:
# Encode the species column
y_encoded = le.fit_transform(df_filtered['species'])
df_filtered['class_encoded'] = y_encoded

# One-hot encode the remaining categorical variables
X = pd.get_dummies(df_filtered.drop(['species', 'class_encoded'], axis=1), drop_first=True)
y = df_filtered['class_encoded']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)

# Predict on the testing data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100 ,"%")
print(logreg.coef_, logreg.intercept_)




Accuracy: 100.0 %
[[ 1.51843077 -1.39670917 -0.14267764 -0.00374337  0.73799007 -0.56451421
  -0.22730594]] [-0.07777031]


<p><strong>Q12. Is it correct to apply label encoding followed by feature scaling on a categorical feature?</strong></p>
<p>Ans: No, this approach is not correct. Label encoding introduces an ordinal relationship that does not exist in categorical data.</p>
<p><strong>Proposed Solution:</strong> Use one-hot encoding instead of label encoding to avoid misleading relationships, and then apply feature scaling if needed.</p>


### Question 2.
#### Q 1: Estimated Probability

Given the logistic regression coefficients $w_0 = -5.9$, $w_1 = 0.06$, and $w_2 = 1.5$, and the variables $x_1 = 50$ hours (hours studied) and $x_2 = 3.6$ (GPA), we calculate the probability ($p$) of receiving an A+ using the logistic regression formula:

$$
p = \frac{1}{1 + e^{-(w_0 + w_1 \cdot x_1 + w_2 \cdot x_2)}}
$$

Substituting the values:

$$
p = \frac{1}{1 + e^{-( -5.9 + 0.06 \cdot 50 + 1.5 \cdot 3.6 )}}
$$

Therefore, the estimated probability is $0.9241$.

---

### Q 2: Hours of Study for a 60% Chance of A+

To determine the number of study hours required to achieve a 60% chance of receiving an A+, we can rearrange the equation to solve for $x_1$ (hours studied) when $p = 0.6$:

$$
0.6 = \frac{1}{1 + e^{-(w_0 + w_1 \cdot x_1 + w_2 \cdot x_2)}}
$$

Solving for $x_1$ gives:

$$
1 + e^{-(w_0 + w_1 \cdot x_1 + w_2 \cdot x_2)} = \frac{1}{0.6}
$$

$$
e^{-(w_0 + w_1 \cdot x_1 + w_2 \cdot x_2)} = \frac{1}{0.6} - 1
$$

Taking the natural logarithm on both sides:

$$
-(w_0 + w_1 \cdot x_1 + w_2 \cdot x_2) = \ln \left( \frac{1}{0.6} - 1 \right)
$$

Now, solving for $x_1$:

$$
0.06 \cdot x_1 = -\ln \left( \frac{1}{0.6} - 1 \right) + 5.9 - 1.5 \cdot 3.6
$$

Finally:

$$
x_1 = \frac{-\ln \left( \frac{1}{0.6} - 1 \right) + 5.9 - 1.5 \cdot 3.6}{0.06}
$$

Therefore, the number of hours a student like the one in part 1 needs to complete is approximately $15.091$.


In [1]:
import numpy as np

# Given coefficients
w0 = -5.9
w1 = 0.06
w2 = 1.5

# Logistic function
def logistic_regression(x1, x2):
    z = w0 + w1 * x1 + w2 * x2
    return 1 / (1 + np.exp(-z))

# Part 1: Probability of getting an A+ for 50 hours of study and GPA of 3.6
x1_study_hours = 50
x2_gpa = 3.6
probability = logistic_regression(x1_study_hours, x2_gpa)
print(f"Estimated probability of receiving an A+: {probability:.4f}")

# Part 2: Solve for the number of study hours required to achieve 60% probability
target_probability = 0.60
x2_gpa = 3.6

# Rearranging the logistic regression formula to solve for x1 (hours studied)
z_target = np.log(target_probability / (1 - target_probability))  # Inverse of logistic function
x1_required = (z_target - w0 - w2 * x2_gpa) / w1
print(f"Number of study hours required for a 60% chance of A+: {x1_required:.2f} hours")


Estimated probability of receiving an A+: 0.9241
Number of study hours required for a 60% chance of A+: 15.09 hours
